In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , classification_report , auc , roc_curve , ConfusionMatrixDisplay ,  roc_auc_score
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.preprocessing import LabelEncoder , RobustScaler , StandardScaler
from sklearn.neural_network import MLPClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD ,Adam

plt.style.use("fivethirtyeight")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/creditcardfraud/creditcard.csv


# **Obtain the Data Set**

In [28]:
creditCard = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [29]:
creditCard.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [30]:
creditCard.sample(25)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
179446,124083.0,-2.080606,-2.920073,0.836205,-3.672559,1.604440,3.699491,-1.557915,1.329677,-1.760050,...,0.314032,0.347930,0.584926,0.640121,0.574636,-0.046452,0.201931,0.171541,272.65,0
284732,172721.0,1.076175,-3.116353,-2.051439,-0.953189,-1.544838,-1.124645,0.385570,-0.698014,-1.829401,...,0.104853,-0.598243,-0.343164,0.088299,-0.267658,-0.313651,-0.132091,0.056029,664.60,0
38754,39537.0,-0.172148,0.341395,0.580698,-0.204648,2.356911,4.223087,-0.045608,0.897343,0.338123,...,-0.146037,-0.099979,-0.308039,1.017950,0.383556,-0.340399,-0.079871,-0.198726,9.00,0
169743,119837.0,2.004663,0.060830,-1.620149,0.355413,0.339289,-0.670483,0.069125,-0.080676,0.281039,...,-0.300814,-0.834183,0.348940,0.659244,-0.329375,0.143142,-0.070244,-0.038328,8.99,0
212124,138725.0,-1.254332,1.148182,0.012605,2.442406,2.594571,0.507687,1.731575,-0.426469,-1.605617,...,-0.146922,0.050381,-0.129169,-0.306776,0.416783,-0.024486,-0.197810,0.105781,37.07,0
155896,106736.0,-0.551360,1.493324,0.746882,0.793177,1.814976,-0.670263,2.514047,-1.771406,2.122837,...,-0.411260,0.634144,-0.426238,-0.049774,0.018470,-0.596653,-0.544272,-0.840479,4.46,0
103759,68780.0,0.491469,-0.917547,1.675188,3.001976,-1.464269,0.749245,-0.702555,0.384311,0.512069,...,0.370712,0.607597,-0.221228,0.405781,0.080558,0.148798,0.022320,0.086863,281.47,0
217211,140807.0,-0.370954,0.522789,0.675353,-0.917137,1.130020,-0.684720,1.052606,-0.257544,-0.246461,...,0.329245,1.070658,-0.468978,-0.377676,0.169085,-0.152158,-0.057305,-0.094756,1.00,0
20502,31088.0,1.125036,-0.036482,-0.178659,0.082547,-0.278707,-1.180716,0.459184,-0.304386,-0.485310,...,0.065345,0.029189,-0.141477,0.631739,0.478797,1.031595,-0.128429,-0.003767,79.90,0
251427,155343.0,0.100963,0.455049,0.128088,-0.374821,0.883196,-0.711429,0.672051,-0.099717,-0.449380,...,-0.119987,-0.332765,0.135141,-0.322723,-0.904309,0.141380,0.067550,0.093148,1.29,0


# **Exploring the Data Set**

In [31]:
creditCard.describe().T

,count,mean,std,min,25%,50%,75%,max
Time,284807.0,9.481386e+04,47488.145955,0.000000,54201.500000,84692.000000,139320.500000,172792.000000
V1,284807.0,1.168375e-15,1.958696,-56.407510,-0.920373,0.018109,1.315642,2.454930
V2,284807.0,3.416908e-16,1.651309,-72.715728,-0.598550,0.065486,0.803724,22.057729
V3,284807.0,-1.379537e-15,1.516255,-48.325589,-0.890365,0.179846,1.027196,9.382558
V4,284807.0,2.074095e-15,1.415869,-5.683171,-0.848640,-0.019847,0.743341,16.875344
V5,284807.0,9.604066e-16,1.380247,-113.743307,-0.691597,-0.054336,0.611926,34.801666
V6,284807.0,1.487313e-15,1.332271,-26.160506,-0.768296,-0.274187,0.398565,73.301626
V7,284807.0,-5.556467e-16,1.237094,-43.557242,-0.554076,0.040103,0.570436,120.589494
V8,284807.0,1.213481e-16,1.194353,-73.216718,-0.208630,0.022358,0.327346,20.007208
V9,284807.0,-2.406331e-15,1.098632,-13.434066,-0.643098,-0.051429,0.597139,15.594995


In [32]:
creditCard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [33]:
print(f"The Data Set contain {creditCard.shape[0]} Rows & {creditCard.shape[1]} Columns")
print(f"The Data Set contain {creditCard.duplicated().sum()} record duplicated") # Flag Here
print(f"Number of Nulls in each features :\n{creditCard.isna().sum()}")

The Data Set contain 284807 Rows & 31 Columns
The Data Set contain 1081 record duplicated
Number of Nulls in each features :
Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


# **Imbalanced Features Resolving**

In [34]:
X , y = creditCard.drop('Class', axis=1) , creditCard['Class']

smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled_smote, y_resampled_smote, test_size=0.2, random_state=42)

scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# **Modeling Our Data**

In [35]:
model = Sequential()

model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))  
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(len(np.unique(y_train)), activation='softmax')) 

model.compile(optimizer=Adam(learning_rate=0.0005), 
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs= 10, batch_size=32, verbose=1)

y_pred_multiclass = np.argmax(model.predict(X_test), axis=1)  

print("Optimized Multiclass Classification Report:")
print(classification_report(y_test, y_pred_multiclass))
print(precision_score(y_test, y_pred_multiclass))
print(f"Accuracy: {accuracy_score(y_test, y_pred_multiclass)}")

model.evaluate(X_test,y_test)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9897 - loss: 0.0325
Epoch 2/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9987 - loss: 0.0060
Epoch 3/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9991 - loss: 0.0042
Epoch 4/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9993 - loss: 0.0032
Epoch 5/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9995 - loss: 0.0032
Epoch 6/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9994 - loss: 0.0031
Epoch 7/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9995 - loss: 0.0023
Epoch 8/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9996 - loss: 0.0019
Epoch 9/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9996 - loss: 0.0022
Epoch 10/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.9995 - loss: 0.0037
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


In [38]:
model = Sequential()

model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))  
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(len(np.unique(y_train)), activation='softmax')) 

model.compile(optimizer=SGD(learning_rate=0.0005), 
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs= 50, batch_size=32, verbose=1)

y_pred_multiclass = np.argmax(model.predict(X_test), axis=1)  

print("Optimized Multiclass Classification Report:")
print(classification_report(y_test, y_pred_multiclass))
print(precision_score(y_test, y_pred_multiclass))
print(f"Accuracy: {accuracy_score(y_test, y_pred_multiclass)}")

model.evaluate(X_test,y_test)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step - accuracy: 0.9086 - loss: 0.3261
Epoch 2/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9693 - loss: 0.0898
Epoch 3/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9776 - loss: 0.0647
Epoch 4/50
 6241/14216 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9815 - loss: 0.0547

KeyboardInterrupt: 